# Computação Bioinspirada 2024-02

Este repositório contém o trabalho da Tópicos Avançados em Ciências de Computação II
(Computação Bioinspirada, no semestre 2024-02).

O trabalho consiste em trabalhar com um conjunto de dados multirrótulo onde cada instância representa uma sequência de
proteína. Cada rótulo (classe) corresponde a uma localização subcelular e as proteínas podem estar presentes
simultaneamente em dois ou mais compartimentos celulares. O conjunto de dados possui seis localizações
subcelulares: Proteínas do Capsídeo Viral, Proteínas da Membrana Celular do Hospedeiro, Proteínas do Retículo
Endoplasmático do Hospedeiro, Proteínas do Citoplasma do Hospedeiro, Proteínas do Núcleo do Hospedeiro e
Proteínas Secretadas. As colunas representam os códigos de Gene Ontology (relacionados à função da proteína),
com valores que indicam a frequência do código para cada proteína. As seis últimas colunas indicam a presença
(1) ou ausência (0) da proteína em cada uma das localizações subcelulares mencionadas.

Desenvolvemos um modelo de classificação multirrótulo usando **redes neurais artificiais**.

Existem dois conjuntos de dados: um de vírus e um de plantas.


### Imports 

In [119]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import random
from copy import deepcopy

In [120]:
random_seed = 42
tf.random.set_seed(random_seed)
tf.keras.utils.set_random_seed(random_seed)

### Lendo os dados dos datasets e separando em treino e teste

In [121]:
plants_dataset = pd.read_csv('./Plants_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]
virus_dataset =  pd.read_csv('./Virus_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]

In [122]:
# Os dois datasets possuem 6 rótulos
NUM_ROTULOS = 6
TAMANHO_TEST = 0.3

# Separar as features e os rótulos para o dataset de plantas
X_plants = plants_dataset.iloc[:, :-NUM_ROTULOS].values  # Todas as colunas, exceto as últimas 6 e a primeira
y_plants = plants_dataset.iloc[:, -NUM_ROTULOS:].values  # As últimas 6 colunas (rótulos)

# Separar as features e os rótulos para o dataset de vírus
X_virus = virus_dataset.iloc[:, :-NUM_ROTULOS].values
y_virus = virus_dataset.iloc[:, -NUM_ROTULOS:].values

# Dividir dados em treino e teste
X_train_plants, X_test_plants, y_train_plants, y_test_plants = train_test_split(
    X_plants, y_plants, test_size=TAMANHO_TEST, random_state=random_seed
)

X_train_virus, X_test_virus, y_train_virus, y_test_virus = train_test_split(
    X_virus, y_virus, test_size=TAMANHO_TEST, random_state=random_seed
)

In [123]:

# Padronizar os dados
scaler_plants = StandardScaler()
X_train_plants = scaler_plants.fit_transform(X_train_plants)
X_test_plants = scaler_plants.transform(X_test_plants)

scaler_virus = StandardScaler()
X_train_virus = scaler_virus.fit_transform(X_train_virus)
X_test_virus = scaler_virus.transform(X_test_virus)

In [124]:
print(f'No dataset das plantas, tenho {len(X_train_plants)} dados de treinamento, {len(X_test_plants)} de teste')
print(f'No dataset dos vírus, tenho {len(X_train_virus)} dados de treinamento, {len(X_test_virus)} de teste')

No dataset das plantas, tenho 675 dados de treinamento, 290 de teste
No dataset dos vírus, tenho 143 dados de treinamento, 62 de teste


In [125]:
def hamming_loss(predictions, actual_values):
    N = len(predictions)
    NUM_ROTULOS = 6

    s = 0

    for i in range(N):
        for j in range(NUM_ROTULOS):
            s += predictions[i][j] ^ actual_values[i][j]
    
    return s / (N * NUM_ROTULOS)

In [126]:
class NeuralNetwork():
    def __init__(self, in_sz, out_sz, h_layer_num, features):
        self.in_sz = in_sz
        self.out_sz = out_sz
        self.h_layer_num = h_layer_num
        self.layer_features = features
        self.model = None
        self.hamming_loss = 0

    def create_model(self):
        model = None
        model = Sequential()
        model.add(Input(shape=(self.in_sz,)))
        self.h_layer_num = 0
        for neur_num, act_func in zip(self.layer_features[0], self.layer_features[1]):
            model.add(Dense(neur_num, activation=act_func))
            self.h_layer_num += 1
        model.add(Dense(self.out_sz, activation='sigmoid'))  # Saída com sigmoid para classificação binária
        
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        self.model = model
    
    def train_model(self, X_train, y_train, epochs, batch, val_split):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch, 
                       validation_split=val_split, verbose=0)
    
    def model_loss(self, X_test, y_test):
        predictions_test = self.model.predict(X_test)
        predicted_test = (predictions_test > 0.5).astype(int)
        self.hamming_loss = hamming_loss(predicted_test, y_test)
        
        

In [127]:
# Criar e treinar o modelo para plantas
features = [[128, 64], ['relu', 'relu']]
nn = NeuralNetwork(X_train_plants.shape[1], y_train_plants.shape[1], 2, features)
nn.create_model()
history_plants = nn.train_model(X_train_plants, y_train_plants, epochs=50, batch=32, val_split=0.3)

In [128]:
# Criar e treinar o modelo para vírus
nn2 = NeuralNetwork(X_train_virus.shape[1], y_train_virus.shape[1], 2, features)
nn2.create_model()
history_virus = nn2.train_model(X_train_virus, y_train_virus, epochs=50, batch=32, val_split=0.3)

In [129]:
# Obter previsões para o conjunto de teste de plantas
predictions_plants_test = nn.model.predict(X_test_plants)

# Obter previsões para o conjunto de treino de plantas
predictions_plants_train = nn.model.predict(X_train_plants)

# Obter previsões para o conjunto de teste de vírus
predictions_virus_test = nn2.model.predict(X_test_virus)

# Obter previsões para o conjunto de treino de vírus
predictions_virus_train = nn2.model.predict(X_train_virus)


# Transformar as probabilidades em rótulos binários (0 ou 1) com um limiar de 0.5
predicted_classes_plants_test = (predictions_plants_test > 0.5).astype(int)
predicted_classes_plants_train = (predictions_plants_train > 0.5).astype(int)
predicted_classes_virus_test = (predictions_virus_test > 0.5).astype(int)
predicted_classes_virus_train = (predictions_virus_train > 0.5).astype(int)


5/5 [==============================] - 0s 2ms/step


In [130]:
possibilidades = [
    [predicted_classes_plants_train, y_train_plants, 'dados de treino das plantas'],
    [predicted_classes_plants_test, y_test_plants, 'dados de teste das plantas'],
    [predicted_classes_virus_train, y_train_virus, 'dados de treino dos vírus'],
    [predicted_classes_virus_test, y_test_virus, 'dados de teste dos vírus']
]

In [131]:
for possibilidade in possibilidades:
    print(f'A hamming loss nos {possibilidade[-1]} foi de {100*hamming_loss(*possibilidade[:-1]):.2f}%')

A hamming loss nos dados de treino das plantas foi de 1.09%
A hamming loss nos dados de teste das plantas foi de 3.91%
A hamming loss nos dados de treino dos vírus foi de 3.38%
A hamming loss nos dados de teste dos vírus foi de 6.72%


In [132]:
# possíveis valores para nossa população inicial
LIM_NUM_NEURONS = 300
LIM_INF_NUM_NEURONS = 16
LIM_NUM_LAYERS = 5
LIST_ACT_FUNCS = ['leaky_relu'] #['relu', 'gelu', 'leaky_relu', 'silu']

p_neur = [i for i in range(LIM_INF_NUM_NEURONS, LIM_NUM_NEURONS+1)]
p_h_layers = [i for i in range(1, LIM_NUM_LAYERS+1)]
p_act_func = LIST_ACT_FUNCS


In [133]:
# gerando uma RN com parâmetros 'aleatórios' (dentro de opções pré definidas)

def gen_random_nn():
    layers = random.choice(p_h_layers)
    act_funcs = [random.choice(p_act_func) for _ in range(layers)]
    neur_num = [random.choice(p_neur) for _ in range(layers)]
    nn = NeuralNetwork(X_train_plants.shape[1], y_train_plants.shape[1], layers, [neur_num, act_funcs])
    return nn
        

In [ ]:
# Inicialização da população
class Particle:
    def __init__(self, nn: NeuralNetwork):
        self.nn = nn
        self.position = {
            "neurons": nn.layer_features[0],
            "activation": nn.layer_features[1],
            'layers': nn.h_layer_num
        }
        self.velocity = {
            "neurons": [random.uniform(-5, 5) for _ in range(nn.h_layer_num)],
            "activation": [random.uniform(-1, 1) for _ in range(nn.h_layer_num)],
            'layers': random.uniform(-1, 1)
        }
        self.best_position = deepcopy(self.position)
        self.best_loss = float("inf")
        self.current_loss = float("inf")

# Função para atualizar a posição e velocidade de cada partícula
def update_particle(particle, global_best_position, w=0.5, c1=1.5, c2=1.5):

    r1, r2 = random.random(), random.random()
    particle.velocity['layers'] = (
            w * particle.velocity["layers"]
            + c1 * r1 * (particle.best_position["layers"] - particle.position["layers"])
            + c2 * r2 * (global_best_position["layers"] - particle.position["layers"])
        )
    particle.position['layers'] = max(
        1, min(LIM_NUM_LAYERS, round(particle.position["layers"] + particle.velocity["layers"]))
    )

    for i in range(particle.nn.h_layer_num):
        # Atualizar velocidade
        r1, r2 = random.random(), random.random()
        particle.velocity["neurons"][i] = (
            w * particle.velocity["neurons"][i]
            + c1 * r1 * ((0 if i >= len(particle.best_position['neurons']) else particle.best_position["neurons"][i]) - particle.position["neurons"][i])
            + c2 * r2 * ((0 if i >= len(global_best_position['neurons']) else global_best_position["neurons"][i]) - particle.position["neurons"][i])
        )
        # Atualizar posição (respeitando os limites)
        particle.position["neurons"][i] = max(
            LIM_INF_NUM_NEURONS, min(LIM_NUM_NEURONS, round(particle.position["neurons"][i] + particle.velocity["neurons"][i]))
        )

    # Ajustar funções de ativação com base na vizinhança
    for i in range(len(particle.position["activation"])):
        # particle.velocity["activation"][i] += (w * random.uniform(-1, 1) + c1 * r1 - c2 * r2)
        particle.position["activation"][i] = random.choice(p_act_func)
    
    dif = particle.position['layers'] - particle.nn.h_layer_num

    if dif > 0:
        for i in range(dif):
            particle.position['neurons'].append(LIM_INF_NUM_NEURONS)
            particle.velocity['neurons'].append(random.uniform(-5, 5))
            particle.position['activation'].append(random.choice(p_act_func))
            particle.velocity['activation'].append(random.uniform(-1, 1))
    elif dif < 0:
        for i in range(-dif):
            particle.position['neurons'].pop()
            particle.velocity['neurons'].pop()
            particle.position['activation'].pop()
            particle.velocity['activation'].pop()
            
        

# Avaliação do fitness (Hamming Loss)
def evaluate_fitness(particle, X_train, y_train, X_test, y_test):
    print(f'My particle will have position={particle.position}')
    print(f'Its nn layer features before were {particle.nn.layer_features}')
    particle.nn.layer_features = [
        particle.position["neurons"],
        particle.position["activation"]
    ]
    particle.nn.create_model()
    particle.nn.train_model(X_train, y_train, epochs=50, batch=32, val_split=0.3)
    particle.nn.model_loss(X_test, y_test)
    particle.current_loss = particle.nn.hamming_loss

    # Atualizar melhor posição
    if particle.current_loss < particle.best_loss:
        particle.best_loss = particle.current_loss
        particle.best_position = particle.position

In [135]:

# Função principal para o PSO
def pso_algorithm(X_train, y_train, X_test, y_test, topology_graph, population_size=12, generations=10):
    # Gerar população inicial
    population = [Particle(gen_random_nn()) for _ in range(population_size)]
    global_best_loss = float("inf")
    global_best_position = None

    for generation in range(generations):
        print(f"Geração {generation + 1}/{generations}")
        fitnesses_and_pos = []

        for particle in population:
            evaluate_fitness(particle, X_train, y_train, X_test, y_test)
            fitnesses_and_pos.append((particle.current_loss, deepcopy(particle.position)))

            # Atualizar melhor global
            if particle.current_loss < global_best_loss:
                global_best_loss = particle.current_loss
                global_best_position = deepcopy(particle.position)
        print(fitnesses_and_pos)
        # Atualizar partículas
        for i, particle in enumerate(population):
            neigs = topology_graph[i]
            melhor_fit, melhor_fit_pos = fitnesses_and_pos[i]
            print(i, neigs, melhor_fit, melhor_fit_pos)
            for neig in neigs:
                if fitnesses_and_pos[neig][0] < melhor_fit:
                    melhor_fit, melhor_fit_pos = fitnesses_and_pos[neig]
            update_particle(particle, melhor_fit_pos)

        #for particle in population:
        #    update_particle(particle, global_best_position)

        print(f"Melhor Hamming Loss da geração {generation}: {global_best_loss:.4f}")

    return global_best_loss, global_best_position

In [136]:
pop_size = 12
ring_graph = [[(i-1) % pop_size, (i+1) % pop_size] for i in range(pop_size)]

In [137]:
pso_algorithm(X_train_plants, y_train_plants, X_test_plants, y_test_plants, ring_graph, population_size=pop_size)

Geração 1/10
My particle will have position={'neurons': [156], 'activation': ['leaky_relu'], 'layers': 1}
Its nn layer features before were [[156], ['leaky_relu']]
10/10 [==============================] - 0s 2ms/step
My particle will have position={'neurons': [127, 135, 274, 29, 117], 'activation': ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu'], 'layers': 5}
Its nn layer features before were [[127, 135, 274, 29, 117], ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu']]
10/10 [==============================] - 0s 2ms/step
My particle will have position={'neurons': [65, 199, 192], 'activation': ['leaky_relu', 'leaky_relu', 'leaky_relu'], 'layers': 3}
Its nn layer features before were [[65, 199, 192], ['leaky_relu', 'leaky_relu', 'leaky_relu']]
10/10 [==============================] - 0s 1ms/step
My particle will have position={'neurons': [164, 56, 135, 67, 210], 'activation': ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu'

(0.034482758620689655,
 {'neurons': [295, 119, 175, 220],
  'activation': ['leaky_relu', 'leaky_relu', 'leaky_relu', 'leaky_relu'],
  'layers': 4})

In [138]:
class individual():
    def __init__(self, nn: NeuralNetwork):
        self.nn = nn
        self.neur_pos = np.zeros(nn.h_layer_num)
        # adicionar posição e velocidades e pensar como isso vai funcionar para evoluir
        

In [139]:
def gen_random_individual():
    nn = gen_random_nn()
    ind = gen_random_individual(nn)

In [ ]:
for ind in pop:
    print(ind.hamming_loss*100)

3.67816091954023
3.67816091954023
4.310344827586207


## O que está feito -> 
Classe NeuralNetwork, que vai ser o indivíduo do nosso 
PSO

## O que precisa fazer? 
O PSO.

### Mais especificamente:

1. Gerar uma população de 12 indivíduos (existem motivos para esse número*) 
aleatórios
2. Inicializar a posição e velocidade
3. Realizar o treinamento de cada indivíduo e analisar a perda de hamming como fitness
4. Realizar a atualização da posição e velocidade de cada indivíduo
5. Voltar para 3 (posição e velocidade mudarão os parâmetros dos ind)

### Considerações sobre o algoritmo:

1. Quantas vizinhanças? Definir as vizinhanças por N de camdas? São poucos indivíduos
então seria computacionalmente barato iterar pela população
2. O split nos dados de treino e teste vai ser feito a cada geração? Um para cada
vizinhança, um para cada indivíduo?

## Quais serão nossos hiperparâmetros? 

Número de Neurônios por camada [32-256] passo de 16

Número de camadas escondidas [2-4]

Tipo da função de ativação [relu, gelu, leakyrelu, tanh]

## Considerações do que pode ser incluso como HPP

Número de Épocas

Tamanho do batch

validation split

??

